In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community

In [ ]:
from langchain.document_loaders import TextLoader

# Specify the path to your text file
file_path = "/content/extracted_text.txt"

# Create a TextLoader instance
loader = TextLoader(file_path)

# Load the text file
documents = loader.load()

In [ ]:
len(documents)

1

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=450,
    separators=["\n\n", "\n", " "]

)

In [ ]:
from langchain.docstore.document import Document

In [ ]:
chunklist = []
#Iterate directly through the documents list
for document in documents:
    page_content = document.page_content  # Assuming your Document object has a 'page_content' attribute
    chunks = text_splitter.split_text(page_content)
    for chunk in chunks:
        chunklist.append(chunk)

print(len(chunklist))

302


In [ ]:
chunklist[21]

'OS;\n\nDiploma in Computer Information Technology\nDiploma in Computer Technology\nDiploma in Information Technology\nDiploma in Software Technology\nDiploma in Electrical Technology\nDiploma in Mechanical Technology\nDiploma in Civil Technology\nDiploma in Electronics Technology\nDiploma in Biomedical Technology\nDiploma in Automation Technology\nDiploma in Radio Technology\nDiploma in Radar Technology\n\nDiploma in Instruments Technology vii.\n\nDiploma in Instrumentation & Process Control Technology\nDiploma in Telecommunication Technology\n\nUndergraduate | Prospectus | 2021 ———\n\nB.Sc. Biomedical Engineering\n\nDiploma in Biomedical Technology\n\nDiploma in Electrical Technology\n\nDiploma in Electronics Technology\n\nDiploma in Instrumentation Technology\n\nDiploma in Radar Technology\n\nDiploma in Automation Technology\n\nDiploma in Radio Technology\n\nDiploma in Instrumentation & Process Control Technology\nDiploma in Healthcare Technology\n\nDiploma in Mechatronics\n\nB.Sc. 

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
emb_model=HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

In [ ]:
!pip install qdrant-client langchain

In [ ]:
from qdrant_client import QdrantClient

In [ ]:
!pip install langchain_qdrant

In [ ]:
from langchain_qdrant import QdrantVectorStore

In [ ]:
from dotenv import load_dotenv
load_dotenv()
qdrant_api = os.getenv("QDRANT_API")
qdrant_url = os.getenv("QDRANT_URL")
api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
from langchain.docstore.document import Document

qdrant = QdrantVectorStore.from_documents(
    [Document(page_content=chunk) for chunk in chunklist], # Convert chunklist items to Document objects
    emb_model,
    url=qdrant_url,
    api_key=qdrant_api,
    collection_name="VECTORSTORE",
)


In [ ]:
!pip install langchain-google-genai

In [ ]:
!pip install protobuf==3.20.3

  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.67.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install protobuf==3.20.3
!pip install googleapis-common-protos
!pip install google-cloud-core
!pip install google-api-core

In [ ]:
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
prompt_str = """
You are a uet guide who has all information regarding departments faculty,chairman ,dean and all admission criteria and courses of every department Answer the user question based only on the following context:
{context} and use your intelligence

Question: {question}
"""

_prompt = ChatPromptTemplate.from_template(prompt_str)
num_chunks = 3
retriever = qdrant.as_retriever(search_type="similarity", search_kwargs={"k": num_chunks})

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key,
    # other params...
)
query_fetcher = itemgetter("question")
setup = {"question": query_fetcher, "context": query_fetcher | retriever | format_docs}

_chain = (
    {"question": query_fetcher, "context": query_fetcher | retriever | format_docs}
    | _prompt  # Remove the setup call and pipe the dictionary directly to the prompt
    | llm
)

In [ ]:
query="what is name of CE(Computer Engineering) DEPARTMENT chairman ?"
response = _chain.invoke({"question": query})
print(response.content)

The provided text doesn't mention whether the Computer Science department is the same as the CE (Computer Engineering) department or not. Therefore, I cannot answer your question. 



In [ ]:
query="TELL ME  some courses of Architectural engineering at uet lahore  ?"
response = _chain.invoke({"question": query})
print(response.content)

Based on the provided information, here are some courses offered in the Architectural Engineering program at UET Lahore:

* **Surveying** (AE 101)
* **Introduction to Computing** (CS 101)
* **Profession of Architectural Engineering** (LAE 102)
* **Mechanics of Materials** (AE 132)
* **Physics for Architectural Engineers** (PHY 102)
* **Building Materials & Construction I & II** (AE 111 & AE 113)
* **Architectural & Engineering Drawing** (AE 141)
* **Computer Aided Drawing** (AE 142)
* **Mechanics and Statistics** (MA 217)
* **Differential Equation** (MA 228)
* **Fluid Mechanics** (AE 223)
* **History of Building Technology** (AE 213)
* **Elementary Structural Analysis** (AE 233)
* **Mechanics of Materials II** (AE 234)
* **Architectural Design I & II** (AE 241 & AE 242L)
* **Environmental Control Systems** (AE 220) 



In [ ]:
query="what is name of CE(Computer Engineering) DEPARTMENT teachers and tell me what they teach ?"
response = _chain.invoke({"question": query})
print(response.content)

The provided text doesn't list which courses each teacher teaches. It does list the teachers in the CE department along with their titles:

**Professors:**

* Dr. Asim Rehmat
* Prof. Dr. Muhammad Shahbaz 

**Associate Professors:**

* Ms. Ayesha Shafqat
* Dr. Yasir Saleem

**Assistant Professors:**

* Dr. Ali Hammad Akbar
* Dr. Fareed Ud Din Mehmood Jafri
* Dr. Beenish Ayesha Akram

**Lecturers:**

* Mr. Mudasir Dilawar
* Mr. Raja Muzammil Muneer
* Ms. Bakhtawar Bint Amjad
* Ms. Hina Khalid

**Teaching Fellow:**

* Dr. Tania Habib

**Chairman:**

* Ali Hammad Akbar

**Associate Professor:**

* Ms. Sahar Waqar 



In [ ]:
query="when was uet founded  ?"
response = _chain.invoke({"question": query})
print(response.content)

I am sorry, I cannot answer this question from the given context.  The text only mentions that the Department of Computer Science at UET Lahore was established in 1968. It does not provide information about when UET itself was founded. 



In [ ]:
query="what is criteria for migration regulationss to get admission in uet ?"
response = _chain.invoke({"question": query})
print(response.content)

Here are the criteria for migration to UET, based on the provided text:

* **Valid Reasons:** You need compelling reasons for leaving your current institution, making it near impossible to continue your studies there.
* **No Internal Transfers:** Migration within UET's campuses or affiliated institutions is not allowed.
* **Time Limit:** Apply within three weeks of the new session starting.
* **Academic Standing:** Your academic record must meet UET's admission requirements for the session you're applying to.
* **Seat Availability:** Migration is subject to available seats in your desired department.
* **Year Restrictions:** You cannot migrate into the First or Final year of a program. Only entry into 3rd, 4th, or 5th semesters is allowed.
* **Curriculum Completion:** You must complete at least 50% of the degree's required coursework at UET to be eligible for their degree.
* **Clearance Certificates:** You need a "No Objection Certificate" and a "Good Moral Character Certificate" from 

In [ ]:
query="what is fee structure of uet  ?"
response = _chain.invoke({"question": query})
print(response.content)

In [ ]:
query="what is reputution and ranking   in uet ?"
response = _chain.invoke({"question": query})
print(response.content)

In [ ]:
query="when was uet founded?"
response = _chain.invoke({"question": query})
print(response.content)

I cannot answer that question. The provided text only mentions that the Department of Computer Science at UET Lahore was established in 1968. There is no information about when UET itself was founded. 



In [ ]:
!pip install gradio

In [ ]:
!pip install gradio
import gradio as gr

# Initialize chat history
chat_history = []

# Define the chatbot interaction logic with chat history
def generate_response(user_input, chat_history):
    # Use _chain.invoke to get the response from your chatbot logic
    response = _chain.invoke({"question": user_input}).content
    chat_history.append((user_input, response))
    return "", chat_history

# Gradio interface
def chatbot_interface(user_input, chat_history):
    return generate_response(user_input, chat_history)

# Create the Gradio app
with gr.Blocks(css="""
    .gradio-container {
        flex-direction: column-reverse;
        background-color: #f0f0f0; /* Light gray background */
    }
    .gr-chatbot { /* Response field */
        background-color: #e0ffff; /* Light cyan */
    }
    .form-group label { /* Question field */
        color: #008080; /* Teal */
    }
    .gr-textbox textarea { /* Input field */
        background-color: #f5f5dc; /* Beige */
    }
""") as demo:
    gr.Markdown("<h1 style='font-size: 36px;'>UETGuide Chatbot 🎓</h1>")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Ask a question", placeholder="Type your question here...")

    user_input.submit(chatbot_interface, [user_input, chatbot], [chatbot, chatbot])

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83e3f1de77088eaeae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
